In [3]:
%pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: lz4
    Found existing installation: lz4 4.3.2
    Uninstalling lz4-4.3.2:
      Successfully uninstalled lz4-4.3.2
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install keras_vggface keras_applications

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install tensorflow keras-applications
%pip install git+https://github.com/rcmalli/keras-vggface.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 4.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
Note: you may need to restart the 

In [14]:
# BMI Prediction API using VGG-Face model

import os
import numpy as np
import pandas as pd
import cv2
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
import tensorflow as tf
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import base64
from io import BytesIO
from PIL import Image

In [15]:
# Import our custom VGGFace implementation
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# Initialize Flask app
app = Flask(__name__)

ImportError: cannot import name 'layer_utils' from 'keras.utils' (/opt/anaconda3/lib/python3.12/site-packages/keras/api/utils/__init__.py)

In [ ]:
# Set up paths
DATA_PATH = "data.csv"
IMAGE_PATH = "images"
MODEL_PATH = "bmi_predictor_model.h5"

# Initialize face detector
detector = MTCNN()

# Define image size (VGG-Face input size)
IMAGE_SIZE = (224, 224)

# Function to load and preprocess the dataset
def load_data(data_path, image_folder):
    """Load data from CSV and preprocess images"""
    # Read the data.csv file
    df = pd.read_csv(data_path)
    
    # Initialize arrays for features and labels
    X = []
    y = []
    
    for idx, row in df.iterrows():
        image_path = os.path.join(image_folder, row['name'])
        
        # Check if image exists
        if not os.path.exists(image_path):
            print(f"Warning: Image {image_path} not found")
            continue
        
        # Load and preprocess image
        img = preprocess_face_image(image_path)
        if img is not None:
            X.append(img)
            y.append(row['bmi'])
    
    return np.array(X), np.array(y)

# Function to detect and preprocess face in an image
def preprocess_face_image(image_path, method='mtcnn'):
    """Detect face in image and preprocess it for VGG-Face"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image: {image_path}")
            return None
        
        # Convert BGR to RGB (VGG-Face expects RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if method == 'mtcnn':
            # Use MTCNN for face detection
            faces = detector.detect_faces(img)
            if not faces:
                print(f"No face detected in {image_path}")
                return None
            
            # Get the largest face
            face = max(faces, key=lambda x: x['box'][2] * x['box'][3])
            x, y, w, h = face['box']
            face_img = img[y:y+h, x:x+w]
            
        else:
            # Use the entire image if not using MTCNN
            face_img = img
        
        # Resize image to the required size for VGG-Face
        face_img = cv2.resize(face_img, IMAGE_SIZE)
        
        # Preprocess for VGG-Face
        face_img = preprocess_input(face_img, version=1)
        
        return face_img
        
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

# Function to preprocess base64 image
def preprocess_base64_image(base64_string):
    """Process base64 image for prediction"""
    try:
        # Decode base64 string
        img_data = base64.b64decode(base64_string)
        img = Image.open(BytesIO(img_data))
        img = np.array(img)
        
        # Convert to RGB if needed
        if len(img.shape) == 2:  # Grayscale
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:  # RGBA
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        
        # Detect face using MTCNN
        faces = detector.detect_faces(img)
        if not faces:
            print("No face detected in the image")
            # Use the entire image if no face is detected
            face_img = cv2.resize(img, IMAGE_SIZE)
        else:
            # Get the largest face
            face = max(faces, key=lambda x: x['box'][2] * x['box'][3])
            x, y, w, h = face['box']
            face_img = img[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, IMAGE_SIZE)
        
        # Preprocess for VGG-Face
        face_img = preprocess_input(face_img, version=1)
        
        return np.expand_dims(face_img, axis=0)
        
    except Exception as e:
        print(f"Error processing base64 image: {str(e)}")
        return None

# Function to create the VGG-Face model for BMI prediction
def create_model():
    """Create and compile the VGG-Face model for BMI prediction"""
    # Load VGG-Face model with pre-trained weights
    base_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add custom regression layers
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='linear')(x)  # BMI is a continuous value
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

# Function to train the model
def train_model(X, y, model, epochs=50, batch_size=4, validation_split=0.2):
    """Train the model with the dataset"""
    history = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        verbose=1
    )
    
    return history

# Function to save model training history as a plot
def save_training_plot(history):
    """Save the training history as a plot"""
    plt.figure(figsize=(12, 4))
    
    # Plot training & validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    # Plot training & validation mean absolute error
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('Model Mean Absolute Error')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history.png')

# Function to predict BMI from an image
def predict_bmi(model, image):
    """Predict BMI from a preprocessed image"""
    prediction = model.predict(image)
    return prediction[0][0]

# Create HTML templates
def create_templates():
    """Create HTML templates for the web interface"""
    os.makedirs('templates', exist_ok=True)
    
    # Create index.html
    with open('templates/index.html', 'w') as f:
        f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>BMI Prediction from Face Image</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .upload-area {
            border: 2px dashed #ccc;
            border-radius: 5px;
            padding: 20px;
            text-align: center;
            margin-bottom: 20px;
            width: 100%;
        }
        .webcam-area {
            margin-top: 30px;
            padding: 20px;
            border: 1px solid #eee;
            border-radius: 5px;
            width: 100%;
        }
        #results {
            margin-top: 20px;
            padding: 20px;
            border: 1px solid #eee;
            border-radius: 5px;
            display: none;
        }
        #videoElement {
            width: 100%;
            max-width: 400px;
            border-radius: 5px;
        }
        button {
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 10px 20px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 10px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
        #preview {
            max-width: 400px;
            max-height: 300px;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>BMI Prediction from Face Image</h1>
        
        <div class="upload-area">
            <h2>Upload Image</h2>
            <input type="file" id="fileInput" accept="image/*">
            <p>or drag and drop image here</p>
            <img id="preview" style="display: none;">
            <button id="uploadButton">Predict BMI</button>
        </div>
        
        <div class="webcam-area">
            <h2>Use Webcam</h2>
            <p>Allow camera access to use this feature</p>
            <video autoplay="true" id="videoElement"></video>
            <div>
                <button id="captureButton">Capture</button>
                <button id="predictButton" disabled>Predict BMI</button>
            </div>
            <canvas id="canvas" style="display: none;"></canvas>
        </div>
        
        <div id="results">
            <h2>Prediction Results</h2>
            <p>Estimated BMI: <span id="bmiValue">--</span></p>
            <p>BMI Category: <span id="bmiCategory">--</span></p>
            <div id="bmiInfo"></div>
        </div>
    </div>
    
    <script>
        // Variables to store images
        let capturedImage = null;
        let uploadedImage = null;
        
        // Elements
        const fileInput = document.getElementById('fileInput');
        const preview = document.getElementById('preview');
        const uploadButton = document.getElementById('uploadButton');
        const video = document.getElementById('videoElement');
        const captureButton = document.getElementById('captureButton');
        const predictButton = document.getElementById('predictButton');
        const canvas = document.getElementById('canvas');
        const results = document.getElementById('results');
        const bmiValue = document.getElementById('bmiValue');
        const bmiCategory = document.getElementById('bmiCategory');
        const bmiInfo = document.getElementById('bmiInfo');
        
        // Initialize webcam
        if (navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
            navigator.mediaDevices.getUserMedia({ video: true })
                .then(function (stream) {
                    video.srcObject = stream;
                })
                .catch(function (error) {
                    console.error("Could not access webcam: ", error);
                    document.querySelector('.webcam-area p').textContent = 
                        "Could not access webcam. Please check permissions.";
                });
        }
        
        // File upload preview
        fileInput.addEventListener('change', function() {
            const file = this.files[0];
            if (file) {
                const reader = new FileReader();
                reader.onload = function(e) {
                    preview.src = e.target.result;
                    preview.style.display = 'block';
                    uploadedImage = e.target.result;
                }
                reader.readAsDataURL(file);
            }
        });
        
        // Upload area drag and drop
        const uploadArea = document.querySelector('.upload-area');
        
        uploadArea.addEventListener('dragover', function(e) {
            e.preventDefault();
            this.style.borderColor = '#4CAF50';
        });
        
        uploadArea.addEventListener('dragleave', function(e) {
            e.preventDefault();
            this.style.borderColor = '#ccc';
        });
        
        uploadArea.addEventListener('drop', function(e) {
            e.preventDefault();
            this.style.borderColor = '#ccc';
            
            const file = e.dataTransfer.files[0];
            if (file && file.type.match('image.*')) {
                fileInput.files = e.dataTransfer.files;
                const reader = new FileReader();
                reader.onload = function(e) {
                    preview.src = e.target.result;
                    preview.style.display = 'block';
                    uploadedImage = e.target.result;
                }
                reader.readAsDataURL(file);
            }
        });
        
        // Capture button
        captureButton.addEventListener('click', function() {
            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            context.drawImage(video, 0, 0, canvas.width, canvas.height);
            capturedImage = canvas.toDataURL('image/png');
            predictButton.disabled = false;
        });
        
        // Upload button
        uploadButton.addEventListener('click', function() {
            if (uploadedImage) {
                predictBMI(uploadedImage, 'upload');
            } else {
                alert('Please upload an image first');
            }
        });
        
        // Predict button (webcam)
        predictButton.addEventListener('click', function() {
            if (capturedImage) {
                predictBMI(capturedImage, 'webcam');
            } else {
                alert('Please capture an image first');
            }
        });
        
        // Predict BMI function
        function predictBMI(imageData, source) {
            // Remove data URL prefix
            const base64Image = imageData.split(',')[1];
            
            // Show loading state
            results.style.display = 'block';
            bmiValue.textContent = 'Calculating...';
            bmiCategory.textContent = '';
            bmiInfo.textContent = '';
            
            // Send to API
            fetch('/predict', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({
                    image: base64Image,
                    source: source
                })
            })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    bmiValue.textContent = 'Error';
                    bmiInfo.textContent = data.error;
                } else {
                    bmiValue.textContent = data.bmi.toFixed(2);
                    bmiCategory.textContent = data.category;
                    
                    // Add BMI explanation
                    let infoText = '';
                    switch (data.category) {
                        case 'Underweight':
                            infoText = 'BMI less than 18.5 indicates underweight. This may be associated with certain health issues.';
                            break;
                        case 'Normal weight':
                            infoText = 'BMI between 18.5 and 24.9 indicates a healthy weight for most adults.';
                            break;
                        case 'Overweight':
                            infoText = 'BMI between 25 and 29.9 indicates overweight. This may increase risk for certain diseases.';
                            break;
                        case 'Obesity':
                            infoText = 'BMI of 30 or higher indicates obesity. This increases risk for many health conditions.';
                            break;
                    }
                    bmiInfo.textContent = infoText;
                }
            })
            .catch(error => {
                console.error('Error:', error);
                bmiValue.textContent = 'Error';
                bmiInfo.textContent = 'Failed to process the image. Please try again.';
            });
        }
    </script>
</body>
</html>
        ''')

# Flask routes
@app.route('/')
def index():
    """Render the main page"""
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    """API endpoint for BMI prediction"""
    try:
        data = request.json
        base64_image = data.get('image')
        source = data.get('source', 'unknown')
        
        if not base64_image:
            return jsonify({'error': 'No image provided'})
        
        # Preprocess the image
        processed_image = preprocess_base64_image(base64_image)
        
        if processed_image is None:
            return jsonify({'error': 'Could not process the image'})
        
        # Make prediction
        model = app.config['model']
        predicted_bmi = float(predict_bmi(model, processed_image))
        
        # Determine BMI category
        category = 'Unknown'
        if predicted_bmi < 18.5:
            category = 'Underweight'
        elif 18.5 <= predicted_bmi < 25:
            category = 'Normal weight'
        elif 25 <= predicted_bmi < 30:
            category = 'Overweight'
        else:
            category = 'Obesity'
        
        return jsonify({
            'bmi': predicted_bmi,
            'category': category,
            'source': source
        })
        
    except Exception as e:
        return jsonify({'error': str(e)})

# Main function to run the application
def main():
    """Main function to prepare the model and run the app"""
    print("Loading and preprocessing data...")
    
    # Create templates
    create_templates()
    
    # Check if model already exists
    if os.path.exists(MODEL_PATH):
        print(f"Loading existing model from {MODEL_PATH}")
        model = load_model(MODEL_PATH, compile=True)
    else:
        # Load and preprocess the dataset
        X, y = load_data(DATA_PATH, IMAGE_PATH)
        
        if len(X) == 0:
            raise ValueError("No valid images found. Please check the data paths.")
        
        print(f"Loaded {len(X)} images for training")
        
        # Create and train the model
        print("Creating and training the model...")
        model = create_model()
        history = train_model(X, y, model)
        
        # Save the model
        print(f"Saving model to {MODEL_PATH}")
        model.save(MODEL_PATH)
        
        # Save training history plot
        save_training_plot(history)
    
    # Store the model in the app config
    app.config['model'] = model
    
    # Run the Flask app
    print("Starting the web server...")
    app.run(debug=False, host='0.0.0.0', port=5000)

if __name__ == "__main__":
    main()